In [8]:
# Using TensorFlow to solve a Logistic Regression problem

import tensorflow as tf
import numpy

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

('Epoch:', '0050', 'cost=', '0.080399320', 'W=', 0.21722122, 'b=', 1.034351)
('Epoch:', '0100', 'cost=', '0.080008611', 'W=', 0.21914664, 'b=', 1.0204996)
('Epoch:', '0150', 'cost=', '0.079662822', 'W=', 0.22095759, 'b=', 1.0074719)
('Epoch:', '0200', 'cost=', '0.079356790', 'W=', 0.22266088, 'b=', 0.99521846)
('Epoch:', '0250', 'cost=', '0.079085976', 'W=', 0.22426279, 'b=', 0.98369449)
('Epoch:', '0300', 'cost=', '0.078846253', 'W=', 0.22576945, 'b=', 0.97285563)
('Epoch:', '0350', 'cost=', '0.078634076', 'W=', 0.22718655, 'b=', 0.96266097)
('Epoch:', '0400', 'cost=', '0.078446314', 'W=', 0.2285191, 'b=', 0.95307523)
('Epoch:', '0450', 'cost=', '0.078280091', 'W=', 0.22977233, 'b=', 0.94405913)
('Epoch:', '0500', 'cost=', '0.078132965', 'W=', 0.23095104, 'b=', 0.93557984)
('Epoch:', '0550', 'cost=', '0.078002706', 'W=', 0.23205963, 'b=', 0.92760456)
('Epoch:', '0600', 'cost=', '0.077887416', 'W=', 0.23310222, 'b=', 0.92010409)
('Epoch:', '0650', 'cost=', '0.077785321', 'W=', 0.234082